In [3]:
# library import
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re

In [111]:
##we start with entering our processed data and then performing necesary options for te same
## Bring in processed data
DataFrame_playlist = pd.read_csv("C:/Users/anshs/Downloads/datafile_processed.csv")



# Dropping unnecessary columns
DataFrame_playlist.drop(columns=["Unnamed: 0",'Unnamed: 0.1'], inplace = True)
DataFrame_playlist.head()

pos    artist_name               track_uri   
0    0  Missy Elliott  0UaMYEvWZi0ZqiDOoHU3YI  \
1   73  Missy Elliott  0UaMYEvWZi0ZqiDOoHU3YI   
2   14  Missy Elliott  0UaMYEvWZi0ZqiDOoHU3YI   
3   42  Missy Elliott  0UaMYEvWZi0ZqiDOoHU3YI   
4    1  Missy Elliott  0UaMYEvWZi0ZqiDOoHU3YI   

                              artist_uri   
0  spotify:artist:2wIVse2owClT7go1WT98tk  \
1  spotify:artist:2wIVse2owClT7go1WT98tk   
2  spotify:artist:2wIVse2owClT7go1WT98tk   
3  spotify:artist:2wIVse2owClT7go1WT98tk   
4  spotify:artist:2wIVse2owClT7go1WT98tk   

                                   track_name   
0  Lose Control (feat. Ciara & Fat Man Scoop)  \
1  Lose Control (feat. Ciara & Fat Man Scoop)   
2  Lose Control (feat. Ciara & Fat Man Scoop)   
3  Lose Control (feat. Ciara & Fat Man Scoop)   
4  Lose Control (feat. Ciara & Fat Man Scoop)   

                              album_uri  duration_ms_x    album_name   
0  spotify:album:6vV5UrXcfyQD1wu4Qo2I9K         226863  The Cookbook  \
1  spotify:album:6vV5UrXcfyQD1wu4Qo2I9K         226863  The Cookbook   
2  spotify:album:6vV5UrXcfyQD1wu4Qo2I9K         226863  The Cookbook   
3  spotify:album:6vV5UrXcfyQD1wu4Qo2I9K         226863  The Cookbook   
4  spotify:album:6vV5UrXcfyQD1wu4Qo2I9K         226863  The Cookbook   

             name  danceability  ...            type                      id   
0      Throwbacks         0.904  ...  audio_features  0UaMYEvWZi0ZqiDOoHU3YI  \
1   w o r k o u t         0.904  ...  audio_features  0UaMYEvWZi0ZqiDOoHU3YI   
2  party playlist         0.904  ...  audio_features  0UaMYEvWZi0ZqiDOoHU3YI   
3       Dance mix         0.904  ...  audio_features  0UaMYEvWZi0ZqiDOoHU3YI   
4            spin         0.904  ...  audio_features  0UaMYEvWZi0ZqiDOoHU3YI   

                                    uri   
0  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI  \
1  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   
2  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   
3  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   
4  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   

                                          track_href   
0  https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...  \
1  https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...   
2  https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...   
3  https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...   
4  https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...   

                                        analysis_url  duration_ms_y   
0  https://api.spotify.com/v1/audio-analysis/0UaM...         226864  \
1  https://api.spotify.com/v1/audio-analysis/0UaM...         226864   
2  https://api.spotify.com/v1/audio-analysis/0UaM...         226864   
3  https://api.spotify.com/v1/audio-analysis/0UaM...         226864   
4  https://api.spotify.com/v1/audio-analysis/0UaM...         226864   

   time_signature  artist_pop   
0               4          74  \
1               4          74   
2               4          74   
3               4          74   
4               4          74   

                                              genres  track_pop  
0  dance_pop hip_hop hip_pop pop pop_rap r&b rap ...         69  
1  dance_pop hip_hop hip_pop pop pop_rap r&b rap ...         69  
2  dance_pop hip_hop hip_pop pop pop_rap r&b rap ...         69  
3  dance_pop hip_hop hip_pop pop pop_rap r&b rap ...         69  
4  dance_pop hip_hop hip_pop pop pop_rap r&b rap ...         69  

[5 rows x 30 columns]

In [71]:
##removinf duplicate entries and also choosing calums on the basis of attributes


# Removing duplicate songs in data
def drop_duplicates(df):
    df['artists_song'] = df.apply(lambda row: row['artist_name']+row['track_name'],axis = 1)
    return df.drop_duplicates('artists_song')

songsDataFrame = drop_duplicates(DataFrame_playlist)

# Choose helpful columns
def Helpful_columns(df):
       return df[['artist_name','id','track_name','danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', "artist_pop", "genres", "track_pop"]]
songsDataFrame = Helpful_columns(songsDataFrame)
songsDataFrame.head()

artist_name                      id   
0       Missy Elliott  0UaMYEvWZi0ZqiDOoHU3YI  \
6      Britney Spears  6I9VzXrHxO9rA9A5euc8Ak   
19            Beyoncé  0WqIKmW4BTrj3eJFmnCKMv   
46  Justin Timberlake  1AWQoqb9bSvzTjaLralEkT   
55             Shaggy  1lzr43nnXAijIGYnCT8M8H   

                                    track_name  danceability  energy  key   
0   Lose Control (feat. Ciara & Fat Man Scoop)         0.904   0.813    4  \
6                                        Toxic         0.774   0.838    5   
19                               Crazy In Love         0.664   0.758    2   
46                              Rock Your Body         0.892   0.714    4   
55                                It Wasn't Me         0.853   0.606    0   

    loudness  mode  speechiness  acousticness  instrumentalness  liveness   
0     -7.105     0       0.1210       0.03110          0.006970    0.0471  \
6     -3.914     0       0.1140       0.02490          0.025000    0.2420   
19    -6.583     0       0.2100       0.00238          0.000000    0.0598   
46    -6.055     0       0.1410       0.20100          0.000234    0.0521   
55    -4.596     1       0.0713       0.05610          0.000000    0.3130   

    valence    tempo  artist_pop   
0     0.810  125.461          74  \
6     0.924  143.040          84   
19    0.701   99.259          86   
46    0.817  100.972          82   
55    0.654   94.759          75   

                                               genres  track_pop  
0   dance_pop hip_hop hip_pop pop pop_rap r&b rap ...         69  
6                         dance_pop pop post-teen_pop         83  
19                                  dance_pop pop r&b         25  
46                                      dance_pop pop         79  
55                              pop_rap reggae_fusion          2

In [73]:
def genre_preprocess(df):
    df['genres_list'] = df['genres'].apply(lambda x: x.split(" "))
    return df
songsDataFrame = genre_preprocess(songsDataFrame)
songsDataFrame['genres_list'].head()

0     [dance_pop, hip_hop, hip_pop, pop, pop_rap, r&...
6                       [dance_pop, pop, post-teen_pop]
19                                [dance_pop, pop, r&b]
46                                     [dance_pop, pop]
55                             [pop_rap, reggae_fusion]
Name: genres_list, dtype: object

In [70]:
def playlist_preprocess(df):
    df = drop_duplicates(df)
    df = Helpful_columns(df)
    df = genre_preprocess(df)
    return df

In [79]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentimentalAnalysis(df, text_col):
  df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

In [80]:
##output after sentiment analysis


sentiment = sentimentalAnalysis(songsDataFrame, "track_name")
sentiment.head(3)

artist_name                      id   
0    Missy Elliott  0UaMYEvWZi0ZqiDOoHU3YI  \
6   Britney Spears  6I9VzXrHxO9rA9A5euc8Ak   
19         Beyoncé  0WqIKmW4BTrj3eJFmnCKMv   

                                    track_name  danceability  energy  key   
0   Lose Control (feat. Ciara & Fat Man Scoop)         0.904   0.813    4  \
6                                        Toxic         0.774   0.838    5   
19                               Crazy In Love         0.664   0.758    2   

    loudness  mode  speechiness  acousticness  instrumentalness  liveness   
0     -7.105     0        0.121       0.03110           0.00697    0.0471  \
6     -3.914     0        0.114       0.02490           0.02500    0.2420   
19    -6.583     0        0.210       0.00238           0.00000    0.0598   

    valence    tempo  artist_pop   
0     0.810  125.461          74  \
6     0.924  143.040          84   
19    0.701   99.259          86   

                                               genres  track_pop   
0   dance_pop hip_hop hip_pop pop pop_rap r&b rap ...         69  \
6                         dance_pop pop post-teen_pop         83   
19                                  dance_pop pop r&b         25   

                                          genres_list subjectivity  polarity  
0   [dance_pop, hip_hop, hip_pop, pop, pop_rap, r&...          low   Neutral  
6                     [dance_pop, pop, post-teen_pop]          low   Neutral  
19                              [dance_pop, pop, r&b]         high  Negative

#### Tf idf vectorizer operations

In [91]:
def tfidf_ohrprep(df, column, new_name): 
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [115]:
# encoding 
subject_ohe = tfidf_ohrprep(sentiment, 'subjectivity','subject')
subject_ohe.iloc[0]

subject|high      False
subject|low        True
subject|medium    False
Name: 0, dtype: bool

In [116]:
# implementation 


tfidf = TfidfVectorizer()
tfidf_Definematrix =  tfidf.fit_transform(songsDataFrame['genres_list'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_Definematrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
genre_df.drop(columns='genre|unknown')
genre_df.reset_index(drop = True, inplace=True)
genre_df.iloc[0]

genre|21st_century_classical    0.0
genre|432hz                     0.0
genre|_hip_hop                  0.0
genre|_roll                     0.0
genre|a_cappella                0.0
                               ... 
genre|zambian_hip_hop           0.0
genre|zhongguo_feng             0.0
genre|zolo                      0.0
genre|zouk                      0.0
genre|zouk_riddim               0.0
Name: 0, Length: 2147, dtype: float64

In [117]:
print(songsDataFrame['artist_pop'].describe())

count    34247.000000
mean        61.916606
std         19.120147
min          0.000000
25%         51.000000
50%         65.000000
75%         76.000000
max        100.000000
Name: artist_pop, dtype: float64


In [95]:
pop = songsDataFrame[["artist_pop"]].reset_index(drop = True)
scaler = MinMaxScaler()
pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns)
pop_scaled.head()

artist_pop
0        0.74
1        0.84
2        0.86
3        0.82
4        0.75

In [118]:
###DEFINING FEATURE SET FUCNTIONS
#OPERATIONS LIKE nomralisation and encoding
def def_FeatureSet(df, float_cols):
    # Tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_Definematrix =  tfidf.fit_transform(df['genres_list'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_Definematrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.drop(columns='genre|unknown') # drop unknown genre
    genre_df.reset_index(drop = True, inplace=True)
    
    # Sentiment analysis
    df = sentimentalAnalysis(df, "track_name")

    # One-hot Encoding
    subject_ohe = tfidf_ohrprep(df, 'subjectivity','subject') * 0.3
    polar_ohe = tfidf_ohrprep(df, 'polarity','polar') * 0.5
    key_ohe = tfidf_ohrprep(df, 'key','key') * 0.5
    mode_ohe = tfidf_ohrprep(df, 'mode','mode') * 0.5

    # Normalization
    # Scale popularity columns
    pop = df[["artist_pop","track_pop"]].reset_index(drop = True)
    scaler = MinMaxScaler()
    pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns) * 0.2 

    # Scale audio columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    # Concanenate all features
    final = pd.concat([genre_df, floats_scaled, pop_scaled, subject_ohe, polar_ohe, key_ohe, mode_ohe], axis = 1)
    
    # Add song id
    final['id']=df['id'].values
    
    return final

In [119]:
#produce the features.



float_cols = songsDataFrame.dtypes[songsDataFrame.dtypes == 'float64'].index.values
songsDataFrame.to_csv("C:/Users/anshs/Downloads/allmusic_Data.csv", index = False)

# Produce features
complete_feature_set = def_FeatureSet(songsDataFrame, float_cols=float_cols)
complete_feature_set.to_csv("C:/Users/anshs/Downloads/AllfeatureData.csv", index = False)
complete_feature_set.head()


genre|21st_century_classical  genre|432hz  genre|_hip_hop  genre|_roll   
0                           0.0          0.0             0.0          0.0  \
1                           0.0          0.0             0.0          0.0   
2                           0.0          0.0             0.0          0.0   
3                           0.0          0.0             0.0          0.0   
4                           0.0          0.0             0.0          0.0   

   genre|a_cappella  genre|abstract_beats  genre|abstract_hip_hop   
0               0.0                   0.0                     0.0  \
1               0.0                   0.0                     0.0   
2               0.0                   0.0                     0.0   
3               0.0                   0.0                     0.0   
4               0.0                   0.0                     0.0   

   genre|accordion  genre|acid_jazz  genre|acid_rock  ...  key|5  key|6   
0              0.0              0.0              0.0  ...    0.0    0.0  \
1              0.0              0.0              0.0  ...    0.5    0.0   
2              0.0              0.0              0.0  ...    0.0    0.0   
3              0.0              0.0              0.0  ...    0.0    0.0   
4              0.0              0.0              0.0  ...    0.0    0.0   

   key|7  key|8  key|9  key|10  key|11  mode|0  mode|1                      id  
0    0.0    0.0    0.0     0.0     0.0     0.5     0.0  0UaMYEvWZi0ZqiDOoHU3YI  
1    0.0    0.0    0.0     0.0     0.0     0.5     0.0  6I9VzXrHxO9rA9A5euc8Ak  
2    0.0    0.0    0.0     0.0     0.0     0.5     0.0  0WqIKmW4BTrj3eJFmnCKMv  
3    0.0    0.0    0.0     0.0     0.0     0.5     0.0  1AWQoqb9bSvzTjaLralEkT  
4    0.0    0.0    0.0     0.0     0.0     0.0     0.5  1lzr43nnXAijIGYnCT8M8H  

[5 rows x 2179 columns]

In [127]:
# Testing playlist of family:  Mom's playlist
playlistDF_test = DataFrame_playlist[DataFrame_playlist['name']=="Mom's playlist"]
playlistDF_test.head()
playlistDF_test.to_csv("C:/Users/anshs/Downloads/Testresult_list.csv")

In [128]:
def generate_feature_playlistdf(complete_feature_set, playlist_df):
    # Find song features in the playlist
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
    # Find all non-playlist song features
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]
    complete_feature_set_playlist_final = complete_feature_set_playlist.drop(columns = "id")
    return complete_feature_set_playlist_final.sum(axis = 0), complete_feature_set_nonplaylist


In [129]:
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_feature_playlistdf(complete_feature_set, playlistDF_test)


In [130]:
def Recommend_generate_playlist(df, features, nonplaylist_features):
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    # Find cosine similarity between the playlist and the complete song set
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    
    return non_playlist_df_top_40


In [131]:
# Create a list of the top 10 suggestions
Suggestions = Recommend_generate_playlist(songsDataFrame, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)
Suggestions.head(10)

C:\Users\anshs\AppData\Local\Temp\ipykernel_26328\287525853.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]


artist_name                      id              track_name   
28834  American Authors  64ybTt8CKxPdeXBNnu08Op                Believer  \
51128  American Authors  1obisQNOcikRvTdStbW3pG       Go Big Or Go Home   
43254          The 1975  51cd3bzVmLAjlnsSZn4ecW          She's American   
28926        Neon Trees  0K1KOCeJBj3lpDYxEX9qP2  Sleeping With A Friend   
54403  American Authors  4gHD93RNqEhEh2NkYzl3x6                    Luck   
55426     WALK THE MOON  71wT7aMCFPYfzutF66OLac                 Aquaman   
44455        Neon Trees  1fBl642IhJOE5U319Gy2Go                  Animal   
43408          The 1975  1v07ywlVYd02pOCnXRBDNA                Menswear   
43278          The 1975  3xrwXWG4O9uhtRyAd3MCou               Heart Out   
14547          The 1975  5hc71nKsUgtwQ3z52KEKQk           Somebody Else   

       danceability  energy  key  loudness  mode  speechiness  acousticness   
28834         0.583   0.968    1    -2.909     1       0.0368      0.001410  \
51128         0.665   0.875    1    -4.272     1       0.0426      0.009390   
43254         0.647   0.857    1    -3.940     1       0.0547      0.167000   
28926         0.582   0.882    2    -4.256     1       0.0355      0.001890   
54403         0.554   0.806    0    -3.463     1       0.0460      0.001770   
55426         0.630   0.772    1    -6.986     1       0.0297      0.510000   
44455         0.482   0.833    5    -5.611     1       0.0449      0.000346   
43408         0.708   0.539    1   -10.281     1       0.0681      0.541000   
43278         0.706   0.830    2    -4.918     1       0.0274      0.008220   
14547         0.610   0.788    0    -5.724     1       0.0585      0.195000   

       ...  liveness  valence    tempo  artist_pop   
28834  ...    0.1300    0.910  119.999          70  \
51128  ...    0.0897    0.660  122.008          70   
43254  ...    0.0763    0.550  115.976          78   
28926  ...    0.3200    0.507  107.034          71   
54403  ...    0.1650    0.646  144.923          70   
55426  ...    0.0881    0.721   99.964          72   
44455  ...    0.3650    0.740  148.039          71   
43408  ...    0.0856    0.159   97.015          78   
43278  ...    0.0763    0.886  118.446          78   
14547  ...    0.1530    0.472  101.045          78   

                                                  genres track_pop   
28834  indie_poptimism modern_alternative_rock modern...        55  \
51128  indie_poptimism modern_alternative_rock modern...        63   
43254       modern_alternative_rock modern_rock pop rock        55   
28926  modern_alternative_rock modern_rock pop pop_ro...        59   
54403  indie_poptimism modern_alternative_rock modern...        54   
55426  dance_pop dance_rock indie_poptimism modern_al...        46   
44455  modern_alternative_rock modern_rock pop pop_ro...        74   
43408       modern_alternative_rock modern_rock pop rock        51   
43278       modern_alternative_rock modern_rock pop rock        54   
14547       modern_alternative_rock modern_rock pop rock        75   

                                             genres_list subjectivity   
28834  [indie_poptimism, modern_alternative_rock, mod...          low  \
51128  [indie_poptimism, modern_alternative_rock, mod...          low   
43254  [modern_alternative_rock, modern_rock, pop, rock]          low   
28926  [modern_alternative_rock, modern_rock, pop, po...          low   
54403  [indie_poptimism, modern_alternative_rock, mod...          low   
55426  [dance_pop, dance_rock, indie_poptimism, moder...          low   
44455  [modern_alternative_rock, modern_rock, pop, po...          low   
43408  [modern_alternative_rock, modern_rock, pop, rock]          low   
43278  [modern_alternative_rock, modern_rock, pop, rock]          low   
14547  [modern_alternative_rock, modern_rock, pop, rock]          low   

      polarity       sim  
28834  Neutral  0.784019  
51128  Neutral  0.781734  
43254  Neutral  0.768755  
28926  Neutral  0.763502  
54403  Neut

In [132]:
playlistDF_test[["artist_name","track_name"]][:20]

artist_name                       track_name
413         The Killers                   Mr. Brightside
1234            Rihanna                    We Found Love
1363   American Authors              Best Day Of My Life
1579       Clean Bandit    Rather Be (feat. Jess Glynne)
1732                Sia                       Chandelier
3986             Hozier                Jackie And Wilson
3999         Aloe Blacc                  I Need a Dollar
4002         Aloe Blacc            Wake Me Up - Acoustic
4007        John Legend                        All of Me
4027  Pharrell Williams   Happy - From "Despicable Me 2"
4043  Justin Timberlake                          Mirrors
4055         Bruno Mars             Locked Out Of Heaven
4067        Mark Ronson                      Uptown Funk
4093      WALK THE MOON                Shut Up and Dance
4120          Matisyahu              Live Like A Warrior
4123        Jess Glynne                     Hold My Hand
4132       Clean Bandit                         Stronger
4134      You Me At Six  Stay With Me - Acoustic Version
4135     Meghan Trainor                   Lips Are Movin
4144        Nico & Vinz                       Am I Wrong